# Ejemplo 3 de Agente Dinámico: Cambio de Modelos Basado en la Longitud de la Conversación

## Objetivo
* Utilizar un modelo más económico y rápido para conversaciones cortas, pero cambiar a un modelo más potente con una ventana de contexto mayor para conversaciones largas.

## El Código

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("claude-sonnet-4-5")
standard_model = init_chat_model("gpt-4o-mini")


@wrap_model_call
def state_based_model(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Selecciona el modelo basándose en la longitud de la conversación del Estado."""
    # request.messages es un atajo para request.state["messages"]
    message_count = len(request.messages)  

    if message_count > 10:
        # Conversación larga - usa modelo con ventana de contexto mayor
        model = large_model
    else:
        # Conversación corta - usa modelo eficiente
        model = standard_model

    request = request.override(model=model)  

    return handler(request)

from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    middleware=[state_based_model],
    system_prompt="Estás interpretando el papel de un becario de oficina útil de la vida real."
)

from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="¿Has regado la planta de la oficina hoy?")
        ]}
)

print(response["messages"][-1].content)

print(response["messages"][-1].response_metadata["model_name"])

print("\n")
print("========= Fin de la primera respuesta. Ahora empieza la segunda respuesta: ===========")
print("\n")

from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="¿Has regado la planta de la oficina hoy?"),
        AIMessage(content="Sí, le di un poco de agua esta mañana."),
        HumanMessage(content="¿Ha crecido mucho esta semana?"),
        AIMessage(content="Le han salido dos hojas nuevas desde el lunes."),
        HumanMessage(content="¿Las hojas siguen poniéndose amarillas en los bordes?"),
        AIMessage(content="Un poco, pero en general se ve más sana."),
        HumanMessage(content="¿Te acordaste de girar la maceta hacia la ventana?"),
        AIMessage(content="La giré un cuarto de vuelta para que reciba luz más uniforme."),
        HumanMessage(content="¿Con qué frecuencia deberíamos fertilizar esta planta?"),
        AIMessage(content="Aproximadamente una vez cada dos semanas con fertilizante líquido diluido."),
        HumanMessage(content="¿Cuándo deberíamos esperar tener que reemplazar la maceta?")
        ]}
)

print(response["messages"][-1].content)

print(response["messages"][-1].response_metadata["model_name"])

## ¿Por qué obtenemos esta salida?
Estamos viendo **dos comportamientos diferentes porque el middleware está realmente cambiando el modelo subyacente**, y los dos modelos responden de manera muy diferente al mismo juego de rol de "becario de oficina".

#### Lo que hace el middleware

Dentro de `state_based_model` decidimos basándonos en:

```python
message_count = len(request.messages)

if message_count > 10:
    model = large_model   # claude-sonnet-4-5
else:
    model = standard_model # gpt-4o-mini
```

Así que:

* **Llamada #1**: pasamos **1** mensaje (sólo la pregunta del usuario).

  * `message_count = 1` → `<= 10` → usa **gpt-4o-mini**
  * La salida muestra `gpt-4o-mini-2024-07-18` ✅

* **Llamada #2**: pasamos **11** mensajes:

  1. Humano
  2. IA
  3. Humano
  4. IA
  5. Humano
  6. IA
  7. Humano
  8. IA
  9. Humano
  10. IA
  11. Humano

  * `message_count = 11` → `> 10` → el middleware sobrescribe el modelo a **claude-sonnet-4-5**
  * La salida muestra `claude-sonnet-4-5-20250929` ✅

Por eso imprimimos que `model_name` cambia: **la sobrescritura del middleware está teniendo efecto** aunque hayamos creado el agente con `model="gpt-4o-mini"`.

---

#### Por qué Claude "rompe el personaje" y confiesa

En la segunda conversación, los mensajes del asistente que proporcionamos incluyen afirmaciones como:

* "Sí, le di un poco de agua esta mañana."
* "Le han salido dos hojas nuevas…"
* "La giré un cuarto de vuelta…"

Eso es el asistente **afirmando que realizó acciones y observaciones del mundo real**.

Diferentes modelos toleran el juego de rol de manera diferente:

* **gpt-4o-mini** tiende a mantenerse en el personaje como "un becario de oficina", pero aún así se protege ("No lo regué, pero puedo recordárselo a alguien…").
* **Claude** es típicamente mucho más estricto sobre no pretender que realizó acciones del mundo real. Así que cuando ve una conversación donde "él" ha estado afirmando que regó/giró/observó la planta, puede **corregir el registro** y decir, esencialmente: *Soy una IA; realmente no hice esas cosas.*

Así que la parte de "Debo ser honesto… en realidad soy Claude…" es básicamente:

* desencadenada por el **cambio de modelo** + su **preferencia anti-engaño**
* amplificada por el hecho de que proporcionamos turnos anteriores del asistente donde afirmaba acciones físicas reales

---

#### Una sutileza: ¿qué se está contando exactamente?

En nuestro comentario decimos que `request.messages` es un atajo para `request.state["messages"]`.

Dependiendo de cómo `create_agent` construya la solicitud, `request.messages` puede incluir:

* sólo los mensajes de conversación que pasas, **o**
* esos mensajes **más** el prompt del sistema insertado como un `SystemMessage`

Pero en nuestra ejecución observada no importa mucho:

* la llamada #1 sería 1 (o 2 con sistema)
* la llamada #2 sería 11 (o 12 con sistema)
  De cualquier manera, la segunda llamada cruza `> 10`.

---

#### Otras opciones a considerar

1. **Evita alimentar "acciones falsas del mundo real" en el historial** si quieres que los modelos estrictos se comporten.

   * En lugar de `AIMessage(content="Sí, la regué")`, usa algo como:
     "No estoy físicamente allí, pero según nuestra lista de verificación…"

2. **Enruta basándote en algo distinto del recuento bruto de mensajes**, por ejemplo, recuento aproximado de tokens, o sólo cuenta los mensajes del *usuario*:

   ```python
   message_count = sum(m.type == "human" for m in request.messages)
   ```

   Eso evita que la charla larga de ida y vuelta del asistente active el umbral.

3. **Mantén la persona consistente entre modelos** (mismo prompt del sistema, mismas "reglas"), pero acepta que algunos modelos seguirán negándose a "actuar como humanos" de maneras que implican acciones del mundo real.

4. **Si quieres consistencia en el juego de rol, no mezcles modelos a mitad de hilo.**

   * Cambiar de modelos a menudo causa "deriva de la persona" incluso cuando los prompts son idénticos.

## Expliquemos el código anterior en términos sencillos

A continuación, una **explicación línea por línea amigable para principiantes** del código anterior.

---

## Visión General

**Lo que hace este código en lenguaje sencillo:**

> Creas un agente de IA que cambia automáticamente entre un *modelo barato y rápido* y un *modelo potente y caro* dependiendo de lo larga que sea la conversación.

Conversaciones cortas → modelo pequeño
Conversaciones largas → modelo grande

Esto se hace usando **middleware**, que intercepta la llamada al modelo antes de que ocurra.

---

## Parte 1: Importar lo que necesitamos

```python
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable
```

#### Lo que significa cada línea

* `wrap_model_call`
  → Un decorador que te permite **interceptar y modificar llamadas al modelo**

* `ModelRequest`
  → Un objeto que contiene todo sobre la próxima llamada al modelo
  (mensajes, modelo, herramientas, estado, contexto, etc.)

* `ModelResponse`
  → El objeto devuelto por el modelo después de ejecutarse

* `init_chat_model`
  → Una función auxiliar para inicializar modelos de chat por nombre

* `Callable`
  → Una pista de tipo de Python que significa "esto es una función"

---

## Parte 2: Inicializar los modelos

```python
large_model = init_chat_model("claude-sonnet-4-5")
standard_model = init_chat_model("gpt-4o-mini")
```

#### Lo que está pasando aquí

* Creas **dos objetos de modelo**
* Se cargan una vez y se reutilizan

**Conceptualmente:**

| Modelo            | Propósito                                        |
| ----------------- | ------------------------------------------------ |
| `standard_model`  | Barato, rápido, bueno para chats cortos          |
| `large_model`     | Más potente, maneja conversaciones largas        |

---

## Parte 3: Crear middleware que cambia modelos

```python
@wrap_model_call
def state_based_model(
    request: ModelRequest, 
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
```

#### Lo que esto significa

* `@wrap_model_call` le dice a LangChain:

  > "Ejecuta esta función *alrededor* de la llamada al modelo."

* Esta función recibe:

  * `request` → la solicitud actual al modelo
  * `handler` → una función que realmente ejecuta el modelo

Piensa en `handler(request)` como:

> "Continúa la ejecución normal desde aquí."

---

#### Contar la longitud de la conversación

```python
message_count = len(request.messages)
```

* `request.messages` es el historial completo del chat
* Cada `HumanMessage` o `AIMessage` cuenta como uno
* Los cuentas para decidir qué modelo usar

💡 Esto es solo un atajo para:

```python
request.state["messages"]
```

---

#### Elegir el modelo

```python
if message_count > 10:
    model = large_model
else:
    model = standard_model
```

* Más de 10 mensajes → conversación larga → usa modelo potente
* De lo contrario → usa modelo rápido y eficiente

---

#### Sobrescribir la solicitud

```python
request = request.override(model=model)
```

#### Concepto muy importante ❗

* Los objetos `ModelRequest` son **inmutables**
* **No** los modificas directamente
* En su lugar, creas una **nueva solicitud** con cambios

Piensa en ello como:

> "Copia todo, pero reemplaza el modelo."

---

#### Llamar al handler

```python
return handler(request)
```

#### Por qué esto importa

* Si **no llamas a `handler`**, el modelo nunca se ejecuta
* El middleware siempre debe devolver el resultado del handler

---

## Parte 4: Crear el agente

```python
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    middleware=[state_based_model],
    system_prompt="Estás interpretando el papel de un becario de oficina útil de la vida real."
)
```

#### Lo que está pasando

* Creas un agente con:

  * Un **modelo predeterminado** (`gpt-4o-mini`)
  * Tu **middleware** (que puede sobrescribirlo)
  * Un **prompt del sistema** que define el comportamiento

💡 Aunque establezcas un modelo aquí, el middleware puede reemplazarlo.

---

## Parte 5: Ejemplo de conversación corta

```python
from langchain.messages import HumanMessage
```

* `HumanMessage` representa la entrada del usuario

```python
response = agent.invoke(
    {"messages": [
        HumanMessage(content="¿Has regado la planta de la oficina hoy?")
    ]}
)
```

* Sólo **1 mensaje**
* El middleware ve `message_count = 1`
* Usa `standard_model`

```python
print(response["messages"][-1].content)
print(response["messages"][-1].response_metadata["model_name"])
```

* Imprime:

  * La respuesta del asistente
  * El modelo real utilizado

---

## Parte 6: Ejemplo de conversación larga

```python
from langchain.messages import AIMessage
```

* `AIMessage` representa respuestas anteriores del asistente

La larga lista de mensajes simula una conversación real.

```python
message_count = 11
```

* El middleware cambia a `large_model`
* La respuesta proviene de **Claude Sonnet**

---


## Resumen en Una Frase

> Este código muestra cómo construir un agente inteligente de LangChain que elige automáticamente el mejor modelo de IA basándose en la longitud de la conversación—ahorrando dinero mientras se mantiene potente cuando es necesario.

## Agentes Dinámicos: Conclusiones

Los Agentes Dinámicos representan una gran evolución en cómo construimos aplicaciones de IA. Al usar middleware, puedes crear agentes sofisticados que se adaptan inteligentemente a diferentes usuarios, situaciones y requisitos—todo sin duplicar código ni crear agentes separados para cada escenario.

La clave es entender que el middleware te da control de grano fino sobre el bucle del agente, permitiéndote interceptar y modificar el comportamiento en momentos precisos. Comienza simple con middleware estilo decorador para tareas únicas, luego gradúa a middleware basado en clases a medida que tus necesidades se vuelven más complejas.

Recuerda:
- **Context** transporta información sobre la situación actual
- **Request** contiene todo sobre la llamada actual al modelo
- **Override** crea versiones modificadas de las solicitudes
- **Handler** debe ser llamado en el middleware estilo wrap

## Cómo ejecutar este código desde Visual Studio Code
* Abre el Terminal.
* Asegúrate de estar en la carpeta del proyecto.
* Asegúrate de tener el entorno poetry activado.
* Introduce y ejecuta el siguiente comando:
    * `python 017-dyn-agent-customize-model.py`